<a href="https://colab.research.google.com/github/ai-project-2023/Speech_grading_multimodal/blob/nlp%2Fspeechtotext/SpeechtoText_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SpeechtoText API

##환경설정

In [ ]:
import pandas as pd
import numpy as np
import io
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
json = 'PATH/keyname.json' #JSON 비공개 키

In [ ]:
!pip install google-cloud-speech

In [ ]:
!gcloud auth login #google-cloud 로그인

In [ ]:
!gcloud config set project projectID # project ID 입력

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

##Audio file to Bucket

In [ ]:
!pip install --upgrade google-cloud-storage

In [ ]:
good_audio_info = pd.read_csv('audio_info_path/.csv') #audio file name, script, label
bad_audio_info = pd.read_csv('audio_info_path/.csv')
good_audio_name = good_audio_info['audio_name'].tolist()
bad_audio_name = bad_audio_info['audio_name'].tolist()

In [ ]:
# 드라이브 폴더 경로 지정
good_folder_path = 'audio_file_path'  # 실제 폴더명으로 수정

# 폴더 내의 모든 파일 이름을 리스트로 저장
good_file_names = os.listdir(good_folder_path)

print(good_file_names)

In [ ]:
for name in good_file_names:
  good_audio_path = good_folder_path+name
  bucket = "bucket_name"
  !gsutil cp "{good_audio_path}" gs://"{bucket}"/

In [ ]:
# 드라이브 폴더 경로 지정
bad_folder_path = 'audio_file_path'  # 실제 폴더명으로 수정

# 폴더 내의 모든 파일 이름을 리스트로 저장
bad_file_names = os.listdir(bad_folder_path)

print(bad_file_names)

In [ ]:
for name in bad_file_names:
  bad_audio_path = bad_folder_path+name
  bucket = "bucket_name"
  !gsutil cp "{bad_audio_path}" gs://"{bucket}"/

##SpeechtoText

In [ ]:
for wav in good_file_names:
  # Import the Google Cloud client library
  from google.cloud import speech_v1p1beta1 as speech

  # Set the environment variable for the Google Cloud project ID
  os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json

  # Create a client instance for the Google Cloud Speech-to-Text API
  client = speech.SpeechClient()

  # Set the URI of the audio file you want to convert
  audio_file_uri = 'bucket_name'+wav

  # Set the audio file format and language code
  config = speech.RecognitionConfig(
      encoding=speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
      sample_rate_hertz=16000,
      language_code='en-US'
  )
  audio = speech.RecognitionAudio(uri=audio_file_uri)

  # Send the audio file URI to the Google Cloud Speech-to-Text API for transcription
  operation = client.long_running_recognize(config=config, audio=audio)
  response = operation.result()

  # Make 1 String variable
  speech_to_text = []
  for result in response.results:
    speech_to_text.append(result.alternatives[0].transcript)
  speech_to_text = '.'.join(speech_to_text) #Divide one line with '.'

  index = good_audio_info[good_audio_info['audio_name'] == wav].index[0]
  good_audio_info.loc[index, 'speech_to_text'] = speech_to_text
  print('finish')

In [ ]:
for i, col in enumerate(good_audio_info.columns):
  print(col, '---',sum(good_audio_info[col].isna()))

In [ ]:
good_audio_info.to_csv('file_path', index=False)

In [ ]:
for wav in bad_file_names:
  # Import the Google Cloud client library
  from google.cloud import speech_v1p1beta1 as speech

  # Set the environment variable for the Google Cloud project ID
  os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json

  # Create a client instance for the Google Cloud Speech-to-Text API
  client = speech.SpeechClient()

  # Set the URI of the audio file you want to convert
  audio_file_uri = 'bucket_name'+wav #ex) gs://audio4770

  # Set the audio file format and language code
  config = speech.RecognitionConfig(
      encoding=speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
      sample_rate_hertz=16000,
      language_code='en-US'
  )
  audio = speech.RecognitionAudio(uri=audio_file_uri)

  # Send the audio file URI to the Google Cloud Speech-to-Text API for transcription
  operation = client.long_running_recognize(config=config, audio=audio)
  response = operation.result()

  # Make 1 String variable
  speech_to_text = []
  for result in response.results:
    speech_to_text.append(result.alternatives[0].transcript)
  speech_to_text = '.'.join(speech_to_text) #Divide one line with '.'

  index = bad_audio_info[bad_audio_info['audio_name'] == wav].index[0]
  bad_audio_info.loc[index, 'speech_to_text'] = speech_to_text
  print('finish')

In [ ]:
for i, col in enumerate(bad_audio_info.columns):
  print(col, '---',sum(bad_audio_info[col].isna()))

In [ ]:
bad_audio_info.to_csv('file_path', index=False)